In [1]:
from tqdm import tqdm
import argparse
from utils import *
from freebase import *
from propagation import *
import random


parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str,
                    default="webqsp", help="choose the dataset from {webqsp, cwq}.")
parser.add_argument("--max_length", type=int,
                    default=1024, help="the max length of LLMs output.")
parser.add_argument("--temperature", type=float,
                    default=0., help="the temperature")
parser.add_argument("--llm", type=str,
                    default="llama", help="choose base LLM model from {llama, gpt-3.5-turbo, gpt-4}.")
parser.add_argument("--openai_api_key", type=str,
                    default="", help="if the LLM is gpt-3.5-turbo or gpt-4, you need add your own openai api key.")
args = parser.parse_args("")


datas, question_string = prepare_dataset(args.dataset)

In [2]:
data = datas[2]
question = data[question_string]
topics = data['topic_entity']
facts, paths = [], []

In [3]:
for topic in topics:
    topic_name = topics[topic]
    relations = get_relations(question, topic, topic_name, args, 3)
    paths.append({"entity:": topic_name, "relations": relations})
    entities_id, entities_name = get_entities(topic, relations)

In [7]:
paths

[{'entity:': 'St. Louis Park',
  'relations': ['location.hud_county_place.county',
   'location.hud_county_place.place',
   'location.statistical_region.unemployment_rate']}]